In [1]:
!python3 -m pip install xmltodict numpy pandas matplotlib mne scikit-learn scipy joblib autoreject tqdm PyWavelets spectrum xgboost seaborn mock plotly

     |████████████████████████████████| 6.4MB 5.8MB/s 
     |████████████████████████████████| 235kB 54.8MB/s 
  Created wheel for spectrum: filename=spectrum-0.7.6-cp36-cp36m-linux_x86_64.whl size=234248 sha256=c2a868a0e6255a350cd8a63dd5f72f3679e4b8dbe0d994f50f587dff32cc0f7c
  Stored in directory: /root/.cache/pip/wheels/7b/a1/1f/16e3bd0418dc16201a4f2e696ab00de3e3c95549cba7df5d13
Successfully built spectrum


In [0]:
#%% #*Import Statements
import os
import sys
import xmltodict
import json
import numpy as np
import pandas as pd
import scipy.stats as sp
import matplotlib.pyplot as plt
import mne
from tqdm import tqdm as tqdm
from spectrum import arburg
from autoreject import AutoReject
import mock
import sklearn.preprocessing as skpr

from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [0]:
DATA_DIR = '/content/drive/My Drive/mahnob/Sessions'

In [0]:
def concat_features(save_as):
  inp = pd.DataFrame()
  sessions = getDataFiles(DATA_DIR)
  for session in tqdm(sessions):
    each = pd.read_pickle(
        os.path.join(DATA_DIR, session['folder'], 'features.pkl'))
    inp = pd.concat([inp, each])
  inp.to_pickle(os.path.join(DATA_DIR, save_as))

In [0]:
def degree2(v, level_names):
  return level_names[0] if v <= 4.5 else level_names[1]


def degree3(v, level_names):
  if v <= 3:
    return level_names[0]
  if v <= 6:
    return level_names[1]
  return level_names[2]

In [0]:
emoDims = ['valence', 'arousal', 'control', 'prediction']
emoDimLabels = [dim + '_label' for dim in emoDims]
test_size = 0.2

In [0]:
def getXGModels(inp, degree_fun, level_names):
  inp2 = inp.copy()
  for idx, emoDim in enumerate(emoDims):
    inp2[emoDimLabels[idx]] = inp2.apply(
        lambda row: degree_fun(row[emoDim], level_names), axis=1)

  inp2 = inp2.drop(columns=emoDims, axis=1)
  train, test = train_test_split(inp2, test_size=test_size)
  train_x = train.drop(columns=emoDimLabels, axis=1)
  test_x = test.drop(columns=emoDimLabels, axis=1)
  #   print('Train X columns')
  #   print(train_x.columns)
  #   print('Test X columns')
  #   print(test_x.columns)
  models = {}
  for emotion in emoDims:
    print('\n\t+++ Model for {}'.format(emotion))
    train_y = train['{}_label'.format(emotion)]
    test_y = test['{}_label'.format(emotion)]
    model = XGBClassifier()
    model.fit(train_x, train_y)
    predict_train = model.predict(train_x)
    accuracy_train = accuracy_score(train_y, predict_train)
    print('accuracy_score on train dataset : ', accuracy_train)
    predict_test = model.predict(test_x)
    accuracy_test = accuracy_score(test_y, predict_test)
    print('accuracy_score on test dataset : ', accuracy_test)
    models[emotion] = model
  for emoDim, emoDimLabel in zip(emoDims, emoDimLabels):
    for degree in level_names:
      frac = inp2[(inp2[emoDimLabel] == degree)].shape[0] / inp2.shape[0]
      print('Fraction of dataset having {} {} = {}'.format(
          degree, emoDim, frac))
  return models, inp2

In [0]:
## Run this for concating the features from each Session
concat_features('data.pkl')

NameError: ignored